# Part2: Model Training

In [132]:
import pandas as pd
import numpy as np
import os
import logging
from data import DATA_DIR
from lama.preprocessing.DataProcessor import nans, reformat_dataframe

logger = logging.getLogger()

In [133]:
PRE_DIR = os.path.join(DATA_DIR, "pre")
train = pd.read_csv(os.path.join(PRE_DIR, "train_groupby.csv"))
test = pd.read_csv(os.path.join(PRE_DIR, "test_groupby.csv"))
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199710 entries, 0 to 199709
Columns: 108 entries, first_active_month to most_recent_purchases_range_nunique
dtypes: float64(61), int64(46), object(1)
memory usage: 164.6+ MB


We do not need card id anynore since we don't have to merge tables

In [134]:
train = train.drop('card_id', axis=1)
train.describe()
features = train.columns.tolist()
features.remove('target')

train = reformat_dataframe(train, features, lambda df: df.fillna(df.mean()))
test = reformat_dataframe(test, features, lambda df: df.fillna(df.mean()))

In [135]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [139]:
PARAMETER_SPACE = {
    "n_estimators": range(10),
    "min_samples_leaf": range(5),
    "min_samples_split": [2, 3],
    "max_depth": [9, 10],
    "max_features": ["auto", 80]
}

In [140]:
clf = RandomForestRegressor(criterion='squared_error', n_jobs=15, random_state=42)
grid = GridSearchCV(clf, PARAMETER_SPACE, cv=2, scoring='neg_mean_squared_error')
print(train['target'].values)
grid.fit(train[features].values, train['target'].values)

[-0.8202826   0.39291325  0.68805599 ...  0.09349415 -4.67658938
 -1.85941301]


KeyboardInterrupt: 

In [138]:
grid.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
np.sqrt(-grid.best_score_)

1.6098819359168235

In [ ]:
test['target'] = grid.best_estimator_.predict(test[features])

KeyError: "['month_lag_nunique_hist_x', 'month_lag_mean_hist_x', 'month_lag_min_hist_x', 'month_lag_max_hist_x', 'month_lag_var_hist_x', 'month_lag_skew_hist_x', 'month_lag_sum_hist_x', 'installments_nunique_hist_x', 'installments_mean_hist_x', 'installments_min_hist_x', 'installments_max_hist_x', 'installments_var_hist_x', 'installments_skew_hist_x', 'installments_sum_hist_x', 'purchase_amount_nunique_hist_x', 'purchase_amount_mean_hist_x', 'purchase_amount_min_hist_x', 'purchase_amount_max_hist_x', 'purchase_amount_var_hist_x', 'purchase_amount_skew_hist_x', 'purchase_amount_sum_hist_x', 'card_id_size_hist_x', 'card_id_count_hist_x', 'authorized_flag_nunique_hist_x', 'category_3_nunique_hist_x', 'category_1_nunique_hist_x', 'merchant_category_id_nunique_hist_x', 'subsector_id_nunique_hist_x', 'merchant_id_nunique_hist_x', 'city_id_nunique_hist_x', 'state_id_nunique_hist_x', 'category_2_nunique_hist_x', 'most_recent_sales_range_nunique_hist_x', 'most_recent_purchases_range_nunique_his

In [ ]:
RESULT_DIR = os.path.join(DATA_DIR, "result")
test[['card_id', 'target']].to_csv(os.path.join(RESULT_DIR, "submission_random_forest_1.csv"), index=False)